In [1]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder

D:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [40]:
with open("训练集/astral_train.fa", "r") as f:  # 打开文件
    data = f.read()  # 读取文件
print(data[0:600])

>d2gkma_ a.1.1.1 (A:) Protozoan/bacterial hemoglobin {Mycobacterium tuberculosis, HbN [TaxId: 1773]}
gllsrlrkrepisiydkiggheaievvvedffvrvladdqlsaffsgtnmsrlkgkqvef
faaalggpepytgapmkqvhqgrgitmhhfslvaghladaltaagvpsetiteilgviap
lavdvts
>d1ngka_ a.1.1.1 (A:) Protozoan/bacterial hemoglobin {Mycobacterium tuberculosis, HbO [TaxId: 1773]}
ksfydavggaktfdaivsrfyaqvaedevlrrvypeddlagaeerlrmfleqywggprty
seqrghprlrmrhapfrislierdawlrcmhtavasidsetlddehrrelldylemaahs
lvnspf
>d2bkma_ a.1.1.1 (A:) automated matches {Geobacillus stearothermophilus [TaxId: 1422]}
eqwqtlyeaiggeetvaklveafyrrvaahpdlrpifpddltetahkqkqfl


In [2]:
# 读取蛋白质的字母序列
l1 = [i.split('}')[1].replace('\n','') for i in data.split('>')[1:]]
# 读取蛋白质的字母id
l2 = [i.split(' ')[1].split('.')[0] for i in data.split('>')[1:]]
# 读取蛋白质的数字id
l3 = [i.split(' ')[1].split('.')[1] for i in data.split('>')[1:]]
# 组合成dataframe
df = pd.DataFrame({'words':l1,'id1':l2,'id2':l3})
df = df.reset_index()

In [3]:
with open("测试集/astral_test.fa", "r") as f:  # 打开文件
    data = f.read()  # 读取文件
l1 = [''.join(i.split('\n')[1:]) for i in data.split('>')[1:]]
l2 = [i.split('\n')[0] for i in data.split('>')[1:]]
df2 = pd.DataFrame({'index':l2,'words':l1})

In [41]:
# 记录蛋白质字母id和数字id的组合
df['id3'] = df['id1']+'.'+df['id2']

In [42]:
# 将训练集和测试集合并
df = pd.concat([df,df2])
# 求字母序列长度
df['len'] = df['words'].apply(lambda x:len(x))

In [44]:
# 记录字母序列中出现过哪些字母
l = []
for i in df['words'].apply(list).values.tolist():
    for j in i:
        l.append(j)
from collections import Counter
c = Counter(l)
print (dict(c))

{'g': 156660, 'l': 202701, 's': 129068, 'r': 111412, 'k': 126220, 'e': 148421, 'p': 98226, 'i': 126880, 'y': 74309, 'd': 127283, 'h': 49943, 'a': 177965, 'v': 155471, 'f': 87382, 'q': 79715, 't': 116158, 'n': 90649, 'm': 47411, 'w': 28814, 'c': 28809, 'X': 285, 'x': 39, 'z': 1}


In [10]:
def w2v_feat(df, tid, fea):
    data_frame=df.copy()
    
    # 将每个人的单词组成一个句子
    df_sens = df['words'].apply(list).values.tolist()
    print('共有',len(df_sens),'人')
    
    # 训练，window=4每个词语周边搜索窗口为4, min_count=2忽略出现次数少于2次的词语,vector_size=10最后产生10维向量
    model = Word2Vec(df_sens, window=4, min_count=1,vector_size=10)
    for i in dict(c):
        print(i,'对应的词向量是',model.wv[i])
    
    tmp = []
    for i in dict(c):
        tmp.append(([i]+model.wv[i].tolist()))
    tmp_df = pd.DataFrame(tmp) # 将每个词的词向量构造成dataframe
    w2c_list = [f'w2c_{fea}_{n}' for n in range(10)]
    tmp_df.columns = [fea] + w2c_list # 修改列名
    print('tmp_df',tmp_df.head(3))
    
    # 将df改造一下方便添加特征。
    df['index2']=df.apply(lambda x:[str(x['index']) for i in range(x.len)],axis=1)
    index0 = []
    for i in df.index2.values.tolist():
        for j in i:
            index0.append(j)
    # 将词向量和每个人的特征merge一下
    df2 = pd.DataFrame({'index':index0,'words':list(''.join(df['words'].values))})
    tmp_df = df2[[tid, fea]].merge(tmp_df, how='left',on=fea)

    # 统计词向量的统计特征
    stat_list = ['mean', 'std','sum']
    # 去除每个人重复词向量，然后计算统计特征
    tmp_df = tmp_df.drop_duplicates().groupby(tid).agg(stat_list).reset_index()
    tmp_df.columns = [tid] + [f'{p}_{q}' for p in w2c_list for q in stat_list]

    return tmp_df

In [17]:
train = w2v_feat(df,'index','words')

共有 11843 人
g 对应的词向量是 [ 0.12083764  0.45871532  0.1438399   0.2627601   0.32339013  0.09466244
  0.8330164   0.16644207  0.45228928 -0.05475282]
l 对应的词向量是 [-0.77510786 -0.26827797  0.2017123  -0.6050412  -0.09990497 -0.3503139
  0.6178934   0.10484625 -0.16002762 -0.3607721 ]
s 对应的词向量是 [ 0.15043202  0.13437702  0.20532253  0.26239255 -0.18978935  0.40872872
  0.6727631  -0.23502304 -0.14302571  0.62639934]
r 对应的词向量是 [-1.1152759  -0.14926873 -0.15826961  0.28426906  0.2963757  -0.14924902
  0.6251291   0.62675256  0.64943653 -0.43715522]
k 对应的词向量是 [ 0.74469924  0.02626805 -0.27263036 -0.7353789  -0.15377544  0.43994942
 -0.32847148  0.7608811  -0.5472175   1.0457903 ]
e 对应的词向量是 [-0.25958318  0.01964059 -0.6957751  -0.02481686  0.3196474   0.28280205
  0.03030771  0.7648866  -0.41090617  0.22621788]
p 对应的词向量是 [-0.30961025 -0.255337   -0.17484339  0.50669277  0.3363199  -0.21364082
  0.8276381  -0.01026826  0.27193126 -0.01848196]
i 对应的词向量是 [ 0.17715262  0.23424366  0.40480405 -1.1400336  

In [18]:
train['index'] = train['index'].astype('str')
df['index'] = df['index'].astype('str')
train = train.merge(df[['index','len','id1','id2','id3']],how='left',on=['index'])

In [20]:
train

,index,w2c_words_0_mean,w2c_words_0_std,w2c_words_0_sum,w2c_words_1_mean,w2c_words_1_std,w2c_words_1_sum,w2c_words_2_mean,w2c_words_2_std,w2c_words_2_sum,...,w2c_words_8_mean,w2c_words_8_std,w2c_words_8_sum,w2c_words_9_mean,w2c_words_9_std,w2c_words_9_sum,len,id1,id2,id3
0,0,0.013099,0.559603,0.235778,-0.078062,0.308304,-1.405121,0.051077,0.383934,0.919385,...,0.028847,0.398730,0.519243,0.239310,0.518660,4.307575,127,a,1,a.1
1,1,-0.011767,0.543080,-0.235345,-0.093941,0.359535,-1.878827,0.019055,0.392033,0.381103,...,0.098546,0.436923,1.970925,0.276888,0.610215,5.537764,126,a,1,a.1
2,10,-0.011767,0.543080,-0.235345,-0.093941,0.359535,-1.878827,0.019055,0.392033,0.381103,...,0.098546,0.436923,1.970925,0.276888,0.610215,5.537764,142,a,1,a.1
3,100,-0.011767,0.543080,-0.235345,-0.093941,0.359535,-1.878827,0.019055,0.392033,0.381103,...,0.098546,0.436923,1.970925,0.276888,0.610215,5.537764,82,a,3,a.3
4,1000,-0.015281,0.557728,-0.290334,-0.053241,0.318554,-1.011584,0.052057,0.373142,0.989092,...,0.057214,0.406746,1.087065,0.314882,0.602142,5.982751,78,a,60,a.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11838,g1gk9.1,0.019443,0.529738,0.388853,-0.120233,0.340746,-2.404651,0.030144,0.396650,0.602877,...,0.051418,0.434469,1.028352,0.197246,0.514560,3.944922,766,NaN,NaN,NaN
11839,g1jjo.1,-0.006536,0.529872,-0.137259,-0.095767,0.350531,-2.011113,0.032028,0.386703,0.672583,...,0.076008,0.438206,1.596174,0.267624,0.596277,5.620098,336,NaN,NaN,NaN
11840,g1k2x.1,-0.006536,0.529872,-0.137259,-0.095767,0.350531,-2.011113,0.032028,0.386703,0.672583,...,0.076008,0.438206,1.596174,0.267624,0.596277,5.620098,292,NaN,NaN,NaN
11841,g1o7,-0.015721,0.541919,-0.314412,-0.112268,0.351170,-2.245357,0.013389,0.386950,0.267779,...,0.095032,0.440603,1.900635,0.247321,0.604274,4.946411,96,NaN,NaN,NaN


In [21]:
# 分离训练和测试集
test = train[train['id1'].isnull()]
train = train[train['id1'].notnull()]
# train['id2'] = train['id2'].astype('int')
test

,index,w2c_words_0_mean,w2c_words_0_std,w2c_words_0_sum,w2c_words_1_mean,w2c_words_1_std,w2c_words_1_sum,w2c_words_2_mean,w2c_words_2_std,w2c_words_2_sum,...,w2c_words_8_mean,w2c_words_8_std,w2c_words_8_sum,w2c_words_9_mean,w2c_words_9_std,w2c_words_9_sum,len,id1,id2,id3
9472,d1a1va1,-0.015281,0.557728,-0.290334,-0.053241,0.318554,-1.011584,0.052057,0.373142,0.989092,...,0.057214,0.406746,1.087065,0.314882,0.602142,5.982751,136,NaN,NaN,NaN
9473,d1a5ea_,-0.051581,0.527122,-0.980044,-0.100268,0.368242,-1.905095,0.034407,0.396551,0.653733,...,0.132534,0.420858,2.518142,0.236420,0.598728,4.491974,156,NaN,NaN,NaN
9474,d1a62a1,-0.018240,0.560308,-0.310081,-0.054832,0.301117,-0.932145,0.072841,0.384135,1.238292,...,0.031637,0.410820,0.537824,0.232067,0.533684,3.945134,47,NaN,NaN,NaN
9475,d1a6ca3,-0.017355,0.557371,-0.329742,-0.080573,0.364244,-1.530882,0.006211,0.398428,0.118003,...,0.086359,0.445390,1.640812,0.280560,0.626709,5.330647,165,NaN,NaN,NaN
9476,d1a79a1,0.003720,0.567535,0.066963,-0.016869,0.284308,-0.303642,0.045457,0.382817,0.818232,...,0.085955,0.398191,1.547186,0.321658,0.618853,5.789847,97,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11838,g1gk9.1,0.019443,0.529738,0.388853,-0.120233,0.340746,-2.404651,0.030144,0.396650,0.602877,...,0.051418,0.434469,1.028352,0.197246,0.514560,3.944922,766,NaN,NaN,NaN
11839,g1jjo.1,-0.006536,0.529872,-0.137259,-0.095767,0.350531,-2.011113,0.032028,0.386703,0.672583,...,0.076008,0.438206,1.596174,0.267624,0.596277,5.620098,336,NaN,NaN,NaN
11840,g1k2x.1,-0.006536,0.529872,-0.137259,-0.095767,0.350531,-2.011113,0.032028,0.386703,0.672583,...,0.076008,0.438206,1.596174,0.267624,0.596277,5.620098,292,NaN,NaN,NaN
11841,g1o7,-0.015721,0.541919,-0.314412,-0.112268,0.351170,-2.245357,0.013389,0.386950,0.267779,...,0.095032,0.440603,1.900635,0.247321,0.604274,4.946411,96,NaN,NaN,NaN


In [45]:
model = lgb.LGBMClassifier(
    eval_metric='auc',
    max_depth=5,
    learning_rate=0.08,
    n_estimators=300,
    gamma=0.1,
    subsample=0.8,
    min_child_weight=1,
    colsample_bytree=0.8,
)

# a = [7, 13, 25, 26, 65, 66, 67, 68]
# x_train.drop(x_train.columns[a], axis=1, inplace=True)
# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train,
#                                                    test_size=0.2, random_state=1)

for pre_fea in ['id1','id2','id3']:
    fea = [i for i in train.columns if i not in ['id1','id2','id3','index']]
    model.fit(train[fea], train[pre_fea])
    test[pre_fea] = model.predict(test[fea])

In [ ]:
test['id4'] = test.apply(lambda x:str(x.id1)+'.'+str(x.id2),axis=1)
test['result'] = test.apply(lambda x:x['id4'] if x['id4'] in df['id3'].unique() else x['id3'],axis=1)
# 判断这个字母id和数字id组合起来是否出现过，如果没出现过，那么就替换

In [ ]:
# 写入

In [51]:
test['result'] = test['id3']

In [48]:
write = test[['index','result']]
write.columns=['sample_id','category_id']
write['sample_id']=write['sample_id'].apply(lambda x:x.replace(' ',''))

In [50]:
write.to_csv('sub.csv',index=False)